In [1]:
!pip install /home/knikaido/work/Cornell-Birdcall-Identification/data/resnest50-fast-package/resnest-0.0.6b20200701/resnest/
!pip install torch==1.4.0
!pip install opencv-python
!pip install slackweb
!pip install torchvision==0.2.2
!pip install torch_summary

Defaulting to user installation because normal site-packages is not writeable
Processing /home/knikaido/work/Cornell-Birdcall-Identification/data/resnest50-fast-package/resnest-0.0.6b20200701/resnest
  Created wheel for resnest: filename=resnest-0.0.6b20200822-py3-none-any.whl size=30753 sha256=f864e9da3fab325bd5c6bca54d129f5bbac1f39cd250d7be283f95d7afd3eac4
  Stored in directory: /tmp/pip-ephem-wheel-cache-2bluy8ev/wheels/98/b8/20/14b175a058326076510265be935570257f33b40bafba7255a9
Successfully built resnest
  Attempting uninstall: resnest
    Found existing installation: resnest 0.0.6b20200822
    Uninstalling resnest-0.0.6b20200822:
      Successfully uninstalled resnest-0.0.6b20200822
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation bec

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from radam import RAdam

from torchsummary import summary


In [3]:
with open('config.yml', 'r') as yml:
    settings = yaml.safe_load(yml)

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
#     torch.backends.cudnn.deterministic = True  # type: ignore
#     torch.backends.cudnn.benchmark = True  # type: ignore
    

In [5]:
# def progress_bar(i):
#     pro_bar = ('=' * i) + (' ' * (pro_size - i))
#     print('\r[{0}] {1}%'.format(pro_bar, i / pro_size * 100.), end='')

In [6]:
# ROOT = Path.cwd().parent
# INPUT_ROOT = ROOT / "input"
INPUT_ROOT = Path("/home/knikaido/work/Cornell-Birdcall-Identification/data")
RAW_DATA = INPUT_ROOT / "birdsong-recognition"
TRAIN_AUDIO_DIR = RAW_DATA / "train_audio"
TRAIN_RESAMPLED_AUDIO_DIRS = [
  INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
]
TEST_AUDIO_DIR = RAW_DATA / "test_audio"

In [7]:
if not TEST_AUDIO_DIR.exists():
    TEST_AUDIO_DIR = INPUT_ROOT / "birdcall-check" / "test_audio"
    test = pd.read_csv(INPUT_ROOT / "birdcall-check" / "test.csv")
else:
    test = pd.read_csv(RAW_DATA / "test.csv")
test.head().T

,0,1,2,3,4
site,site_1,site_1,site_1,site_1,site_1
row_id,site_1_41e6fe6504a34bf6846938ba78d13df1_5,site_1_41e6fe6504a34bf6846938ba78d13df1_10,site_1_41e6fe6504a34bf6846938ba78d13df1_15,site_1_41e6fe6504a34bf6846938ba78d13df1_20,site_1_41e6fe6504a34bf6846938ba78d13df1_25
seconds,5,10,15,20,25
audio_id,41e6fe6504a34bf6846938ba78d13df1,41e6fe6504a34bf6846938ba78d13df1,41e6fe6504a34bf6846938ba78d13df1,41e6fe6504a34bf6846938ba78d13df1,41e6fe6504a34bf6846938ba78d13df1


In [11]:
device = torch.device(settings["globals"]["device"])

In [12]:
class ResNet(nn.Module):
    def __init__(self, base_model_name: str, pretrained=True,
                 num_classes=264):
        super().__init__()
        base_model = models.__getattribute__(base_model_name)(
            pretrained=pretrained)
        layers = list(base_model.children())[:-2]
        layers.append(nn.AdaptiveMaxPool2d(1))
        self.encoder = nn.Sequential(*layers)

        in_features = base_model.fc.in_features

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, num_classes))

    def forward(self, x):
        batch_size = x.size(0)
        x = self.encoder(x).view(batch_size, -1)
        x = self.classifier(x)
        return x
#         multiclass_proba = F.softmax(x, dim=1)
#         multilabel_proba = torch.sigmoid(x)
#         return {
#             "logits": x,
#             "multiclass_proba": multiclass_proba,
#             "multilabel_proba": multilabel_proba
#         }

In [13]:
def get_model(args: tp.Dict):
    model = ResNet('resnet50')
    
    return model

In [14]:
model = get_model(settings["model"])
model = model.to(device)


In [18]:
summary(model, (3, 224, 547))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Conv2d: 2-1                       [-1, 64, 112, 274]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 274]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 274]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 137]         --
|    └─Sequential: 2-5                   [-1, 256, 56, 137]        --
|    |    └─Bottleneck: 3-1              [-1, 256, 56, 137]        75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 56, 137]        70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 56, 137]        70,400
|    └─Sequential: 2-6                   [-1, 512, 28, 69]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 28, 69]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 28, 69]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 28, 69] 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Conv2d: 2-1                       [-1, 64, 112, 274]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 274]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 274]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 137]         --
|    └─Sequential: 2-5                   [-1, 256, 56, 137]        --
|    |    └─Bottleneck: 3-1              [-1, 256, 56, 137]        75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 56, 137]        70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 56, 137]        70,400
|    └─Sequential: 2-6                   [-1, 512, 28, 69]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 28, 69]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 28, 69]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 28, 69] 

In [21]:
!pip install efficientnet-pytorch 　#Google colabだとこれで動きます
# from efficientnet_pytorch import EfficientNet

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: '#Google'
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [22]:
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
# load pretrained models, using ResNeSt-50 as an example
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/user/.cache/torch/hub/master.zip
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
Downloading: "https://s3.us-west-1.wasabisys.com/resnest/torch/resnest50-528c19ca.pth" to /home/user/.cache/torch/checkpoints/resnest50-528c19ca.pth
100%|██████████| 105M/105M [00:17<00:00, 6.26MB/s]


In [50]:
class ResNeSt(nn.Module):
    def __init__(self, base_model_name: str, pretrained=True,
                 num_classes=264):
        super().__init__()
        torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
        # load pretrained models, using ResNeSt-50 as an example
        base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        layers = list(base_model.children())[:-2]
        layers.append(nn.AdaptiveMaxPool2d(1))
        self.encoder = nn.Sequential(*layers)

        in_features = base_model.fc.in_features

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, num_classes))

    def forward(self, x):
        batch_size = x.size(0)
        x = self.encoder(x).view(batch_size, -1)
        x = self.classifier(x)
        return x
#         multiclass_proba = F.softmax(x, dim=1)
#         multilabel_proba = torch.sigmoid(x)
#         return {
#             "logits": x,
#             "multiclass_proba": multiclass_proba,
#             "multilabel_proba": multilabel_proba
#         }

In [51]:
def get_model_new(args: tp.Dict):
    model = ResNeSt('resnet50')
    
    return model

In [52]:
model_new = get_model_new(settings["model"])
model_new = model_new.to(device)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/user/.cache/torch/hub/master.zip
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [53]:
summary(model_new, (3, 224, 547))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Sequential: 2-1                   [-1, 64, 112, 274]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 112, 274]        864
|    |    └─BatchNorm2d: 3-2             [-1, 32, 112, 274]        64
|    |    └─ReLU: 3-3                    [-1, 32, 112, 274]        --
|    |    └─Conv2d: 3-4                  [-1, 32, 112, 274]        9,216
|    |    └─BatchNorm2d: 3-5             [-1, 32, 112, 274]        64
|    |    └─ReLU: 3-6                    [-1, 32, 112, 274]        --
|    |    └─Conv2d: 3-7                  [-1, 64, 112, 274]        18,432
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 274]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 274]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 137]         --
|    └─Sequential: 2-5                   [-1, 256, 56, 137]        --
|    |

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Sequential: 2-1                   [-1, 64, 112, 274]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 112, 274]        864
|    |    └─BatchNorm2d: 3-2             [-1, 32, 112, 274]        64
|    |    └─ReLU: 3-3                    [-1, 32, 112, 274]        --
|    |    └─Conv2d: 3-4                  [-1, 32, 112, 274]        9,216
|    |    └─BatchNorm2d: 3-5             [-1, 32, 112, 274]        64
|    |    └─ReLU: 3-6                    [-1, 32, 112, 274]        --
|    |    └─Conv2d: 3-7                  [-1, 64, 112, 274]        18,432
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 274]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 274]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 137]         --
|    └─Sequential: 2-5                   [-1, 256, 56, 137]        --
|    |

In [1]:
import sys

sys.path.append("./GPU-Efficient-Networks")

In [2]:
import GENet


In [12]:
import torch
from torch import nn
arch = 'GENet_small'
if arch == 'GENet_large':
    input_image_size = 256
    model_ge = GENet.genet_large(pretrained=True, root='./GPU-Efficient-Networks/GENet_params/')
if arch == 'GENet_normal':
    input_image_size = 192
    model_ge = GENet.genet_normal(pretrained=True, root='./GPU-Efficient-Networks/GENet_params/')
if arch == 'GENet_small':
    input_image_size = 192
    model_ge = GENet.genet_small(pretrained=True, root='./GPU-Efficient-Networks/GENet_params/')

AttributeError: type object 'GENet' has no attribute 'genet_small'

In [7]:
class GENet(nn.Module):
    def __init__(self, base_model, pretrained=True,
                 num_classes=264):
        super().__init__()
#         torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
#         # load pretrained models, using ResNeSt-50 as an example
#         base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        layers = list(base_model.children())[:-2]
        layers.append(nn.AdaptiveMaxPool2d(1))
        self.encoder = nn.Sequential(*layers)

        in_features = base_model.fc.in_features

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
            nn.Linear(1024, num_classes))

    def forward(self, x):
        batch_size = x.size(0)
        x = self.encoder(x).view(batch_size, -1)
        x = self.classifier(x)
        return x
#         multiclass_proba = F.softmax(x, dim=1)
#         multilabel_proba = torch.sigmoid(x)
#         return {
#             "logits": x,
#             "multiclass_proba": multiclass_proba,
#             "multilabel_proba": multilabel_proba
#         }

In [8]:
def get_model_ge(model_ge):
    model = GENet(model_ge)
    
    return model

In [9]:
model_new_ge = get_model_ge(model_ge)
model_new_ge = model_new_ge.to(device)

NameError: name 'model_ge' is not defined